# Ejercicio: clasificando dígitos con redes convolucionales

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/lenet.png" style="width:900px;">

En este ejercicio revisitamos el problema de clasificación de dígitos manuscritos, en esta ocasión empleando redes neuronales convolucionales. Veremos cómo esta arquitectura nos permite obtener niveles más altos de acierto.

## Guía general

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb).

Vamos a fijar las semillas aleatorias de numpy y tensorflow para obtener resultados reproducibles entre varias ejecuciones del notebook

In [1]:
import numpy as np
import tensorflow as tf
np.random.seed(1)
tf.random.set_seed(2)

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Carga de datos

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Carga y prepara los datos como hiciste en el notebook anterior. En particular, los pasos que necesitas repetir son:
    <ul>
        <li>Carga los datos usando la función `mnist.load_data` de `tensorflow.keras.datasets`.</li>
        <li>Normaliza los valores de los píxeles de entrada, dividiéndolos por 255, tanto para train como para test.</li>
        <li>Codifica los datos de salida como vectores one-hot, tanto para train como para test.</li>
    </ul>
    De momento <b>no es necesario que hagas reshape de los datos</b> para convertirlos en vectores 1-dimensionales.
</font>

***

In [2]:
####### INSERT YOUR CODE HERE
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train_norm = X_train.astype('float32') / 255
X_test_norm = X_test.astype('float32') / 255
Y_train = to_categorical(y_train, 10) # We have 10 classes to codify
Y_test = to_categorical(y_test, 10)

El resto del notebook asume que has cargado correctamente tus imágenes de entrenamiento como **X_train_norm**, etiquetas de entrenamiento como **Y_train**, imágenes de test como **X_test_norm** y etiquetas de test como **Y_test**.

## Imports de Keras

Necesitaremos importar la siguientes clases de Keras, que ya conoces del notebook anterior.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

## Redes neuronales convolucionales

Para mejorar en este problema de clasificación de imágenes necesitamos tratar los datos como verdaderas imágenes, y tener en cuenta la proximidad entre píxeles para tomar las decisiones, en lugar de "aplanar" todos los píxeles y meterlos a una red neuronal densa. Las capas **Convolucionales** y de **Pooling** son las ideales para ello.

### Formateando los datos como tensores

Así como en el notebook anterior aplanamos los datos para poder introducirlos en nuestras redes, para las redes convolucionales necesitaremos organizar los datos en la forma de un tensor 4-dimensional. Los dimensiones de este tensor representan lo siguiente:
* El índice de la imagen (ej. tercera imagen del dataset)
* El índice de la fila
* El índice de la columna
* El índice del canal (ej. canal de color rojo en imágenes a color)

Nuestros datos ahora mismo tienen la siguiente forma:

In [4]:
X_train_norm.shape

(60000, 28, 28)

Así que, una vez más, tendremos que hacer uso de la función reshape para transformar los datos al formato adecuado. Tenemos 60000 imágenes en nuestros datos de entrenamiento, y esas imágenes tienen 28 filas por 28 columnas. Dado que estas imágenes son en escala de grises, la dimensión del canal solo contiene un canal:

In [5]:
traintensor = X_train_norm.reshape(60000, 28, 28, 1)
traintensor.shape

(60000, 28, 28, 1)

Ahora los datos están en la forma correcta.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Repite la transformación para los datos de test. Ten en cuenta que en test solo contamos con 10000 imágenes. Guarda el tensor resultante en una variable llamada <b>testtensor</b>.
</font>

***

In [6]:
####### INSERT YOUR CODE HERE
testtensor = X_test_norm.reshape(10000, 28, 28, 1)

### Capas de convolución y de pooling

Cuando definimos una red convolucional, las capas de convolución y de pooling trabajan juntas. La forma más habitual de utilizar estas capas es con el siguiente patrón:
* Capa convolucional con activación ReLU
* Capa de Pooling

Siguiendo este patrón, podemos definir una red convolucional mínima como

In [7]:
from tensorflow.keras.layers import Convolution2D, MaxPooling2D

img_rows = 28
img_cols = 28
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

convnet = Sequential()

convnet.add(Convolution2D(
    32, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 1), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
)) 
convnet.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

Pero hay un problema: en algún punto debemos convertir los datos tensoriales a datos "planos" en forma de vector, ya que la salida final de la red debe ser un vector de 10 valores, representando probabilidades de clase. Podemos hacer esto mediante una capa `Flatten`. Tras ella, podemos añadir la habitual capa `Dense` para producir las salidas de la red:

In [8]:
from tensorflow.keras.layers import Flatten
convnet.add(Flatten())
convnet.add(Dense(10, activation="softmax"))

Comprobemos qué tipo de red hemos creado:

In [9]:
convnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Compila la red que hemos definido, escogiendo "adam" como optimizador, y entrénala con los datos de train en su versión tensorial. Emplea un tamaño de batch de 128 y 20 épocas de entrenamiento. Tras entrenar, mide el accuracy sobre el conjunto de test. ¿Han resultado de utilidad las nuevas capas?
</font>

***

In [10]:
####### INSERT YOUR CODE HERE
convnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
convnet.fit(
    traintensor, # Training data
    Y_train, # Labels of training data
    batch_size=128, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    verbose=2 # Level of verbosity of the log messages
)
score = convnet.evaluate(testtensor, Y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/20
469/469 - 2s - loss: 0.3466 - accuracy: 0.9048
Epoch 2/20
469/469 - 2s - loss: 0.1299 - accuracy: 0.9637
Epoch 3/20
469/469 - 2s - loss: 0.0912 - accuracy: 0.9743
Epoch 4/20
469/469 - 2s - loss: 0.0722 - accuracy: 0.9797
Epoch 5/20
469/469 - 2s - loss: 0.0617 - accuracy: 0.9820
Epoch 6/20
469/469 - 2s - loss: 0.0538 - accuracy: 0.9845
Epoch 7/20
469/469 - 2s - loss: 0.0482 - accuracy: 0.9858
Epoch 8/20
469/469 - 2s - loss: 0.0441 - accuracy: 0.9870
Epoch 9/20
469/469 - 2s - loss: 0.0395 - accuracy: 0.9885
Epoch 10/20
469/469 - 2s - loss: 0.0369 - accuracy: 0.9888
Epoch 11/20
469/469 - 2s - loss: 0.0336 - accuracy: 0.9901
Epoch 12/20
469/469 - 2s - loss: 0.0313 - accuracy: 0.9908
Epoch 13/20
469/469 - 2s - loss: 0.0281 - accuracy: 0.9919
Epoch 14/20
469/469 - 2s - loss: 0.0262 - accuracy: 0.9925
Epoch 15/20
469/469 - 2s - loss: 0.0241 - accuracy: 0.9933
Epoch 16/20
469/469 - 2s - loss: 0.0226 - accuracy: 0.9937
Epoch 17/20
469/469 - 2s - loss: 0.0204 - accuracy: 0.9945
Epoch 

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Construye y entrena una red convolucional más grande, con las siguientes capas:
<ul>
     <li>Convolución de 32 canales, tamaño de kernel 3, activación ReLU</li>
     <li>Otra convolución de 32 canales, tamaño de kernel 3, activación ReLU</li>
     <li>MaxPooling de tamaño 2</li>
     <li>Flatten</li>
     <li>Dense de 128 unidades, con activación ReLU</li>
     <li>Dropout del 50%</li>
     <li>Dense de salida con activación softmax</li>
</ul>
¿Has conseguido mejores resultados con esta red más compleja?
</font>

***

In [11]:
####### INSERT YOUR CODE HERE
img_rows = 28
img_cols = 28
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

large_convnet = Sequential()

large_convnet.add(Convolution2D(32, # Number convolution channels to generate
                        (kernel_size, kernel_size),
                        padding='valid',
                        input_shape=(img_rows, img_cols, 1),
                        activation="relu"))
large_convnet.add(Convolution2D(32, (kernel_size, kernel_size), activation="relu"))
large_convnet.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
large_convnet.add(Flatten())
large_convnet.add(Dense(128, activation="relu"))
large_convnet.add(Dropout(0.5))
large_convnet.add(Dense(10, activation="softmax"))

large_convnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
large_convnet.fit(
    traintensor, # Training data
    Y_train, # Labels of training data
    batch_size=128, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    verbose=1 # Level of verbosity of the log messages
)
score = large_convnet.evaluate(testtensor, Y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/20
469/469 [==============================] - 4s 9ms/step - loss: 0.2485 - accuracy: 0.9240
Epoch 2/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0803 - accuracy: 0.9762
Epoch 3/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0609 - accuracy: 0.9822
Epoch 4/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0479 - accuracy: 0.9854
Epoch 5/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0408 - accuracy: 0.9876
Epoch 6/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0350 - accuracy: 0.9889
Epoch 7/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0289 - accuracy: 0.9912
Epoch 8/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0268 - accuracy: 0.9915
Epoch 9/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0250 - accuracy: 0.9919
Epoch 10/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0205 - accuracy: 0.9930

## LeNet

La <a href=http://yann.lecun.com/exdb/lenet/>LeNet</a> es una arquitectura particular de red convolucional que ha demostrado ser particularmente efectiva para este problema. Como ejercicio final, vamos a construir una red similar a LeNet.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Construye y entrena la siguiente red:
<ul>
     <li>Convolución de 32 canales, tamaño de kernel 5, activación ReLU</li>
     <li>MaxPooling de tamaño 2</li>
     <li>Convolución de 50 canales, tamaño de kernel 5, activación ReLU</li>
     <li>MaxPooling de tamaño 2</li>
     <li>Flatten</li>
     <li>Dense de 256 unidades, con activación ReLU</li>
     <li>Dropout del 50%</li>
     <li>Dense de salida con activación softmax</li>
</ul>
¿Es esta el mejor resultado que has obtenido? 
</font>

***

In [12]:
####### INSERT YOUR CODE HERE
img_rows = 28
img_cols = 28

lenet = Sequential()

lenet.add(Convolution2D(
    32,
    (5, 5),
    padding='valid',
    input_shape=(img_rows, img_cols, 1),
    activation="relu"
))
lenet.add(MaxPooling2D(pool_size=2, strides=2))
lenet.add(Convolution2D(50, (5, 5), activation="relu"))
lenet.add(MaxPooling2D(pool_size=2, strides=2))
lenet.add(Flatten())
lenet.add(Dense(256, activation="relu"))
lenet.add(Dropout(0.5))
lenet.add(Dense(10, activation="softmax"))

lenet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
lenet.fit(
    traintensor, # Training data
    Y_train, # Labels of training data
    batch_size=128, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    verbose=1 # Level of verbosity of the log messages
)
score = lenet.evaluate(testtensor, Y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

Epoch 1/20
469/469 [==============================] - 3s 7ms/step - loss: 0.2323 - accuracy: 0.9284
Epoch 2/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0657 - accuracy: 0.9802
Epoch 3/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0483 - accuracy: 0.9854
Epoch 4/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0368 - accuracy: 0.9885
Epoch 5/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0295 - accuracy: 0.9907
Epoch 6/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0254 - accuracy: 0.9920
Epoch 7/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0210 - accuracy: 0.9935
Epoch 8/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0185 - accuracy: 0.9942
Epoch 9/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0170 - accuracy: 0.9944
Epoch 10/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0148 - accuracy: 0.9954

## Bonus track

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***

<font color=#259b4c>
    Entrena la red anterior durante más épocas. ¿Cuál es el mejor acierto en test que puedes obtener?
</font>

***